# Toy example: 
In this notebook all model paremeters, i.e:  noise variance, hyper-parameters of activations and components, variational distributions, are learned simultaneously by maximazing the marginal likelihood.

In [ ]:
%matplotlib inline
import sys 
sys.path.append('../../../')
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import gpflow, gpitch
reload(gpitch)
from gpitch.amtgp import logistic
from scipy.fftpack import fft, ifft, ifftshift
from scipy import signal

In [ ]:
gpitch.amtgp.init_settings()
plt.rcParams['figure.figsize'] = (16, 3)  # set plot size

In [ ]:
np.random.seed(2)
Nc = 10  # number of partials
var = 0.25*np.random.rand(Nc)  # variances
leng = 0.5*np.random.rand()  # lengthscales
f0 = 220.0
freq = f0*np.asarray(range(1, Nc+ 1)) + np.sqrt(100.)*np.random.randn(Nc) # frequencies
kern1 = gpitch.kernels.MaternSpecMixSL(input_dim=1, lengthscales=leng, variances=var, frequencies=freq)
kern2 = gpflow.kernels.Matern32(input_dim=1, lengthscales=0.05, variance=10.)

In [ ]:
f0*np.asarray(range(1, Nc+ 1))

In [ ]:
print freq
print leng
print var

In [ ]:
N, fs = 2400, 16e3
x = np.linspace(0., (N-1.)/fs, N).reshape(-1, 1)
Kcom = kern1.compute_K_symm(x)
Kact = kern2.compute_K_symm(x)

In [ ]:
noise_var = 1e-5
com = np.random.multivariate_normal(np.zeros((N)), Kcom, 1).reshape(-1, 1)
com /= np.max(np.abs(com))
g = np.random.multivariate_normal(np.zeros((N)), Kact, 1).reshape(-1, 1)
act = logistic(g)
act /= np.max(act)
y = act*com + np.sqrt(noise_var)*np.random.randn(N,1)

plt.figure(figsize=(16, 12))
plt.subplot(3,1,1), plt.title('data')
plt.plot(x, y)
plt.subplot(3,1,2), plt.title('activation')
plt.plot(x, act)
plt.subplot(3,1,3), plt.title('component')
plt.plot(x, com)

# New component covariance function

In [ ]:
N = y.size
Y = fft(y.reshape(-1,)) #  FFT data
S =  2./N * np.abs(Y[0:N//2]) #  spectral density data
F = np.linspace(0, fs/2., N//2) #  frequency vector


aux1 = np.ones((Nc, 1))
aux2 = f0*np.asarray(range(1, Nc+1))

aux3 = np.ones((Nc + 1, 1))
aux4 = f0*np.asarray(range(1, Nc+2)) - 0.5*f0

plt.figure()
plt.plot(F, S/np.max(np.abs(S)))
plt.stem(aux2, aux1 )
plt.plot(aux4, aux3, 'xr' )
plt.xlim([0, 2000])


In [ ]:
win = signal.hann(6)
Ss = signal.convolve(S, win, mode='same') / sum(win)

plt.figure(figsize=(16, 6))
plt.subplot(2,1,1)
plt.plot(F, S / np.max(S))
plt.stem(aux2, aux1, 'r', markerfmt=" " )
plt.stem(np.asarray(freq), aux1, 'g' ,markerfmt=" ")
plt.xlim([0, 2000])

plt.subplot(2,1,2)
plt.plot(F, Ss / np.max(Ss), lw=2)
plt.stem(aux2, aux1, 'r', markerfmt=" " )
plt.stem(np.asarray(freq), aux1, 'g' ,markerfmt=" ")
plt.plot(aux4, aux3, 'xC1' )
plt.xlim([0, 2000])

In [ ]:
F_star, S_star= gpitch.amtgp.init_com_params(y=y, fs=fs, Nh=Nc, ideal_f0=f0, scaled=True)
plt.figure()
plt.plot(F, Ss)
plt.plot(F_star, S_star, 'xk', mew=2)
plt.xlim([0, 2000])

In [ ]:
print var

In [ ]:
kern1

In [ ]:
kern2

In [ ]:
kern_com = gpitch.kernels.MaternSpecMixSL(input_dim=1, lengthscales=1., variances=S_star, frequencies=F_star, Nc=1)
kern_act = gpflow.kernels.Matern32(1)

In [ ]:
ws, dec = N//1, 80  # samples per window, decimation factor
m = gpitch.modpdet.ModPDet(x=x, y=y, kern_com=kern_com, kern_act=kern_act, ws=ws, dec=dec, whiten=True)

m.model.kern1.fixed = True
m.model.kern1.lengthscales.fixed = False
m.model.kern1.lengthscales.transform = gpflow.transforms.Logistic(0., 0.1)
m.model.kern1.frequency_1.fixed = False
m.model.kern1.frequency_2.fixed = False
m.model.kern1.frequency_3.fixed = False
m.model.kern1.frequency_4.fixed = False
m.model.kern1.frequency_5.fixed = False
m.model.kern1.frequency_6.fixed = False
m.model.kern1.frequency_7.fixed = False
m.model.kern1.frequency_8.fixed = False
m.model.kern1.frequency_9.fixed = False
m.model.kern1.frequency_10.fixed = False
m.model.kern2.fixed = False
m.model.likelihood.noise_var.fixed = False

In [ ]:
maxiter, restarts = 500, 5
init_hyper, learnt_hyper, mse = m.optimize_restart(maxiter=maxiter, restarts=restarts)

In [ ]:
m.model.kern2.lengthscales = learnt_hyper[0].mean().copy()
m.model.kern2.variance = learnt_hyper[1].mean().copy()
m.model.likelihood.noise_var = learnt_hyper[2].mean().copy()
m.model.kern1.lengthscales = learnt_hyper[3].mean().copy()

m.model.optimize(disp=1, maxiter=500)

In [ ]:
m.model.kern1.fixed = True
m.model.kern2.fixed = True
m.model.likelihood.noise_var.fixed = True
m.model.optimize(disp=1, maxiter=500)

In [ ]:
print freq

In [ ]:
m.model.kern1

In [ ]:
m.model.kern2

In [ ]:
print m.model.likelihood.noise_var

In [ ]:
mean_f, var_f = m.model.predict_f(x)
mean_g, var_g = m.model.predict_g(x)
mean_act = logistic(mean_g)

mean_f = mean_f.reshape(-1,)
var_f = var_f.reshape(-1,)
mean_g = mean_g.reshape(-1,)
var_g = var_g.reshape(-1,)
x_plot = x.reshape(-1,).copy()


xla = 0.0
xlb = 0.15
plt.figure(figsize=(16, 12))
plt.subplot(3,1,1), plt.title('data')
plt.plot(x_plot, y, '.k') 
plt.twinx()
plt.plot(x_plot, logistic(mean_g)*mean_f, lw=2)
plt.xlim([xla, xlb])

plt.subplot(3,1,2), plt.title('activation')

plt.plot(x_plot[::dec], act[::dec], '.k')
plt.twinx()
plt.plot(x_plot, mean_act, 'C1', lw=2)
plt.fill_between(x_plot, logistic(mean_g-2*np.sqrt(var_g)), logistic(mean_g+2*np.sqrt(var_g)), color='C1', alpha=0.2)
plt.xlim([xla, xlb])

plt.subplot(3,1,3), plt.title('component')
plt.plot(x_plot, com, '.k')
plt.twinx()
plt.plot(x_plot, mean_f, 'C2', lw=2)
plt.fill_between(x_plot, mean_f-2*np.sqrt(var_f), mean_f+2*np.sqrt(var_f), color='C2', alpha=0.2)
plt.xlim([0.04, 0.05])

In [ ]:
x_eval = x.copy() 
k_plot_model = m.model.kern1.compute_K(x_eval, np.asarray(0.).reshape(-1,1))
k_plot_sample = kern1.compute_K(x_eval, np.asarray(0.).reshape(-1,1))

plt.figure(figsize=(16, 4))
plt.plot(x_eval, k_plot_model, lw=2)
plt.xlim([0., 0.15])

In [ ]:
N = x_eval.size
Yk1 = fft(k_plot_model.reshape(-1,)) #  FFT data
Sk1 =  2./N * np.abs(Yk1[0:N//2]) #  spectral density data

plt.figure(figsize=(16, 6))
plt.subplot(2, 1, 1)
plt.plot(F, S / np.max(np.abs(S)), lw=2)
plt.legend([' Spectral density data'])
plt.xlabel('Frequency (Hz)')
plt.xlim([0, 3000])

plt.subplot(2, 1, 2)
plt.plot(F, Sk1 / np.max(np.abs(Sk1)), lw=2)
plt.legend([' Spectral density learned kernel'])
plt.xlabel('Frequency (Hz)')
plt.xlim([0, 3000])